<a href="https://colab.research.google.com/github/alejandro-munoz-pujol/blowfish_cipher/blob/main/Blowfish_Cipher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Blowfish cipher (ECB and CBC)





In [ ]:
import os

Reads S-box values from text files (sbox1.txt, sbox2.txt, etc.) and stores them in a list to be used in the encryption and decryption processes.

In [ ]:
def read_sboxes():
  S = []

  for i in range(4):
    with open('sbox'+ str(i+1) +'.txt', 'r') as file:
      lines = file.readlines()

    sbox = []
    for line in lines:
      if line.strip() and not line.strip().startswith("S-box"):
        sbox.extend([int(value, 16) for value in line.split()])

    S.append(sbox)

  return S

In [ ]:
P = [
      0x243F6A88, 0x85A308D3, 0x13198A2E, 0x03707344,
      0xA4093822, 0x299F31D0, 0x082EFA98, 0xEC4E6C89,
      0x452821E6, 0x38D01377, 0xBE5466CF, 0x34E90C6C,
      0xC0AC29B7, 0xC97C50DD, 0x3F84D5B5, 0xB5470917,
      0x9216D5D9, 0x8979FB1B
  ]

S = read_sboxes()

Generates round subkeys derived from the main key (key). These keys are adjusted through multiple encryption steps using an initial data block.

In [ ]:
def generate_keys():
  for i in range(18):
      P[i] = P[i] ^ key[i %len(key)]

  x = 0
  data = 0
  for i in range(0,9):
      temp = encryption(data)
      P[x] = temp >> 32
      x += 1
      P[x] = temp & 0xffffffff
      x += 1
      data = temp

  return P

Implements the encryption process of the algorithm. It splits the input block into two halves (L and R) and processes them through 16 rounds of transformations based on the subkeys and S-boxes.

In [ ]:
def encryption(input):
  L = input >> 32
  R = input & 0xffffffff

  for i in range(16):
    L ^= P[i]
    L1 = f_function(L)
    R ^= f_function(L1)
    L, R = R, L

  L, R = R, L
  L ^= P[17]
  R ^= P[16]

  return (L << 32) ^ R


Reverses the encryption process. It follows the 16 rounds in reverse order, using the same subkeys, to recover the original data.

In [ ]:
def decryption(input):
  L = input >> 32
  R = input & 0xffffffff

  for i in range(17, 1, -1):
    L = P[i]^L
    L1 = f_function(L)
    R ^= f_function(L1)
    L, R = R, L

  L, R = R, L
  L ^= P[0]
  R ^= P[1]

  return (L << 32) ^ R

A core function used in each encryption and decryption round. It transforms the input (L) using the S-box values and basic arithmetic and bitwise operations.

In [ ]:
def f_function(L):
  temp = S[0][L >> 24]
  temp = (temp + S[1][(L >> 16) & 0xFF]) & 0xFFFFFFFF
  temp = temp ^ S[2][(L >> 8) & 0xFF]
  temp = (temp + S[3][L & 0xFF]) & 0xFFFFFFFF

  return temp


Encrypts a list of data blocks using Electronic Codebook (ECB) mode, where each block is encrypted independently.

In [ ]:
def ecb_encrypt(data):
  encrypted_blocks = []
  for block in data:
    encrypted_blocks.append(encryption(block))
  return encrypted_blocks

Decrypts a list of data blocks encrypted in ECB mode by reversing the encryption process for each block.

In [ ]:
def ecb_decrypt(data):
  decrypted_blocks = []
  for block in data:
    decrypted_blocks.append(decryption(block))
  return decrypted_blocks

Encrypts a list of data blocks using Cipher Block Chaining (CBC) mode. Each block is XORed with the previous ciphertext block (or IV for the first block) before encryption.

In [ ]:
def cbc_encrypt(data, iv):
  encrypted_blocks = []
  prev_block = iv
  for block in data:
    block ^= prev_block
    encrypted_block = encryption(block)
    encrypted_blocks.append(encrypted_block)
    prev_block = encrypted_block
  return encrypted_blocks

Decrypts data encrypted in CBC mode by reversing the process: decrypting each block and XORing it with the previous ciphertext block (or IV for the first block).

In [ ]:
def cbc_decrypt(data, iv):
  decrypted_blocks = []
  prev_block = iv
  for block in data:
    decrypted_block = decryption(block)
    decrypted_block ^= prev_block
    decrypted_blocks.append(decrypted_block)
    prev_block = block
  return decrypted_blocks

In [ ]:
key_text = 'h7bss8ksksi'
key = key_text.encode('utf-8')

P = generate_keys()

data_input = int(input("Enter input data: "), 16)

if data_input.bit_length() > 64:
    print("Input too big!")
else:
  data = [data_input]

  mode = input("Choose ciphering mode(ECB o CBC): ").strip().upper()

  if mode == "ECB":
    print("\n--- Mode ECB ---")
    encrypted = ecb_encrypt(data)
    print("Cipher data:", " ".join(f"{block:x}" for block in encrypted))
    decrypted = ecb_decrypt(encrypted)
    print("PLain data:", " ".join(f"{block:x}" for block in decrypted))

  elif mode == "CBC":
    iv = int.from_bytes(os.urandom(8), 'big')
    print("Initialization Vector (IV):", hex(iv))
    print("\n--- Mode CBC ---")
    encrypted = cbc_encrypt(data, iv)
    print("Cipher data:", " ".join(f"{block:x}" for block in encrypted))
    decrypted = cbc_decrypt(encrypted, iv)
    print("Plain data:", " ".join(f"{block:x}" for block in decrypted))

  else:
    print("Invalid mode")


Enter input data: 2e3dd
Choose ciphering mode(ECB o CBC): ECB

--- Mode ECB ---
Cipher data: ead5038b53f6edb0
PLain data: 2e3dd
